In [1]:
import pandas as pd
import pingouin as pg
from statsmodels.stats.multitest import multipletests
import numpy as np
from scipy.stats import *

In [3]:
# Delete genes if half or more of them have a 0 value and if not turn to NaN values
def delete_0s(df):
    deleted_cols = df.columns[(df == 0).sum() >= len(df)/2]
    df = df.drop(columns = deleted_cols)
    print(f'Columnas eliminadas {deleted_cols}')if len(deleted_cols) else None
    # df = df.replace(0, np.nan)
    return df
    
def delete_outliers(df):
    
    means = df.mean()
    std_devs = df.std()
    
    for column in df.columns:
        mean = means[column]
        std_dev = std_devs[column]     
        
        # Turn to 0 outliers, an outlier is over/below the mean+2*desv
        df.loc[(df[column] > mean + 2 * std_dev) | (df[column] < mean - 2 * std_dev), column] = np.nan
        #print(column)

def metrics(df_control, df_treatment):
    p_values_ttest, normality_c, normality_t, homoscedasticity, p_values_anova = [], [], [], [], []
    
    for column in df_control.columns:
        control_values = df_control[column].dropna().tolist()
        treatment_values = df_treatment[column].dropna().tolist()
        
        # t- student P-value
        # result = pg.ttest(control_values.dropna().tolist(), treatment_values.dropna().tolist(), correction=False)
        result = pg.ttest(control_values, treatment_values, correction=False)
        p_value = result['p-val'].iloc[0] 
        p_values_ttest.append(p_value)

        # normality by groups
        _, p_value = shapiro(control_values)
        normality_c.append(p_value)
        _, p_value = shapiro(treatment_values)
        normality_t.append(p_value)

        # homocedasticity
        _, p_value = levene(control_values, treatment_values)
        homoscedasticity.append(p_value)

        # ANOVA
        _, p_value = f_oneway(control_values, treatment_values)
        p_values_anova.append(p_value)
    

    # Adjusted p-values
    # adjusted_p_values_ttest = multipletests(p_values_ttest, method='fdr_bh')[1]
    # adjusted_p_values_anova = multipletests(p_values_anova, method='fdr_bh')[1]
    
    result_df = pd.DataFrame({
        'Normality_control': normality_c,
        'Normality_PD': normality_t,
        'Homocedasticity': homoscedasticity,
        'P-value ANOVA': p_values_anova,
        # 'Adjusted_p-value ANOVA': adjusted_p_values_anova,
        'P-value T-test': p_values_ttest,
        # 'Adjusted_p-value T-test': adjusted_p_values_ttest,
        'Fold_change': df_treatment.mean(axis=0)/df_control.mean(axis=0),
        'Mean_control': df_control.mean(axis=0),
        'Mean_PD': df_treatment.mean(axis=0),
        'Std_control': df_control.std(axis=0),
        'Std_PD': df_treatment.std(axis=0),
    })
    result_df = result_df.transpose()
    return(result_df)

def normalize_columns(df):
    for col in df.columns:
        df[col] = np.log10(df[col]+1)
        # mask = ~df[col].isnull()
        # transformed_values, _ = boxcox(df[col][mask]) 
        # df[col][mask] = transformed_values
    return df

In [4]:
df= pd.read_excel('PATH', sheet_name='Data_4')
df.set_index('Sample', inplace=True)
df = df.drop(columns=['Type'])
df = df.replace(0, np.nan)
# Preprocess
control = df.loc[df['Disease'].str.startswith('C')]
treated = df.loc[df['Disease'].str.startswith('P')]
control = control.drop(columns=['Disease'])
treated = treated.drop(columns=['Disease'])
# print( treated, control)

# Delete outliers
delete_outliers(control)
delete_outliers(treated)

# Delete mewtabolites if half or more of them have a 0 value and if not turn to NaN values
control = delete_0s(control)
treated = delete_0s(treated)

# print(control, treated)

# Metabolites' names without outliers
col_names_no_OLs = control.columns.intersection(treated.columns).tolist()

print(len(col_names_no_OLs))

# Delete columns removed from the other df
df_control_no_OLs = control.loc[:, control.columns.isin(col_names_no_OLs)]
df_treated_no_OLs = treated.loc[:, treated.columns.isin(col_names_no_OLs)]

# Normalize
df_control_no_OLs = normalize_columns(df_control_no_OLs.copy())
df_treated_no_OLs = normalize_columns(df_treated_no_OLs.copy())
# print(df_control_no_OLs, df_treated_no_OLs)

# Join dfs
df_no_OLs =  pd.concat([df_control_no_OLs, df_treated_no_OLs])
# df_no_OLs.to_excel('data_no_OLs.xlsx')
# print(df_no_OLs)

# Get metrics
metrics = metrics(df_control_no_OLs,df_treated_no_OLs)
results = pd.concat([df_no_OLs, metrics])
# print(results)


col_names = results.columns[(results.loc['Fold_change'] >= 1.3) | (results.loc['Fold_change'] <= 1/1.3) ]
col_names = results.columns[(results.loc['Fold_change'] >= 1.15) | (results.loc['Fold_change'] <= 1/1.15) ]
col_names = results.columns
print(len(col_names))

results_filtered = results[col_names]
p_values_anova = results_filtered.loc['P-value ANOVA'].tolist()
p_values_ttest = results_filtered.loc['P-value T-test'].tolist()
adjusted_p_values_anova = multipletests(p_values_anova, method='fdr_bh')[1]
adjusted_p_values_ttest = multipletests(p_values_ttest, method='fdr_bh')[1]

results_filtered = results_filtered.copy()
results_filtered.loc['Adjusted_p-value ANOVA'] = adjusted_p_values_anova
results_filtered.loc['Adjusted_p-value T-test'] = adjusted_p_values_ttest

print(results_filtered)

results_filtered.to_excel('log10+1.xlsx')
#'''

49
49
                               C0        C2        C3  C3-DC (C4-OH)  \
c1                       0.888990  0.306544  0.051067       0.002454   
c2                       0.924279  0.027078  0.016755       0.001734   
c3                       0.941180  0.336460  0.088727       0.003029   
c4                       0.927712  0.264030  0.056269       0.002742   
c5                       0.944483  0.270291  0.073963       0.002310   
c6                       0.976197  0.308920  0.060824       0.003604   
c7                            NaN  0.139879  0.029519       0.001590   
c8                       1.022566  0.304634  0.067443       0.003029   
c9                            NaN  0.551247  0.123688       0.004536   
c10                      1.042707  0.414973  0.112884       0.002814   
c11                      0.827839  0.268237  0.057251       0.004321   
RF_137                   1.077368  0.382917  0.103804       0.002922   
RF_138                   0.975432  0.413300  0.057000     

In [54]:
df= pd.read_excel('PATH', sheet_name='Data_3')
df.set_index('Sample', inplace=True)
old = df.loc[df['Type'] == 'VIEJO']
new = df.loc[df['Type'] == 'NUEVO']

new_control = new.loc[df['Disease'].str.startswith('C')]
new_control = new_control.drop(columns=['Disease', 'Type'])
new_treated = new.loc[df['Disease'].str.startswith('P')]
new_treated = new_treated.drop(columns=['Disease', 'Type'])
old_control = old.loc[df['Disease'].str.startswith('C')]
old_control = old.drop(columns=['Disease', 'Type'])
old_treated = old.loc[df['Disease'].str.startswith('P')]
old_treated = old.drop(columns=['Disease', 'Type'])


result_df = pd.DataFrame({
        'Mean_control_new': new_control.mean(axis=0),
        'Mean_control_old': old_control.mean(axis=0),
        'Mean_PD_new': new_treated.mean(axis=0),
        'Mean_PD_old': old_treated.mean(axis=0),
        'Std_control_new': new_control.std(axis=0),
        'Std_control_old': old_control.std(axis=0),
        'Std_PD_new': new_treated.std(axis=0),
        'Std_PD_old': old_treated.std(axis=0),
    })
result_df = result_df.transpose()
result_df.to_excel('Means_desv.xlsx')
